# [Baseline] 2. Normal equation을 이용한 너비예측
제목에서도 보다시피 방정식을 풀어보도록 하겠습니다. 

## 패키지 설치
pandas는 csv를 불러오고 데이터 프레임을 조작하기 위해 사용됩니다.<br/>
numpy는 데이터 프레임을 행렬로 변환하고 행렬곱, 전치행렬 만들기, 역행렬 계산 등을 하기 위해 불러옵니다.<br/>
sklearn은 python의 대표적인 머신러닝 패키지 입니다. 다양한 머신러닝 알고리즘이 이미 구현되어 있어 우리의 수고를 덜어줄 것입니다.

## 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

iris = pd.read_csv('iris_train.csv')
iris.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


우리가 사용해야 할 데이터는 붓꽃의 종류(species), 꽃잎의 길이(petal length (cm)), 꽃받침의 길이(sepal length (cm))입니다.<br/>
꽃잎과 꽃받침의 너비를 알아내야 하니 X로 정의하겠습니다.

In [2]:
X = iris[['species','sepal length (cm)','petal length (cm)']]
X.head()

,species,sepal length (cm),petal length (cm)
0,setosa,4.4,1.4
1,versicolor,6.4,4.5
2,virginica,6.2,4.8
3,virginica,7.2,6.1
4,setosa,4.9,1.4


정답인 y값도 정의해야겠죠?<br/>
y가 꽃잎과 꽃받침의 너비 2개이므로 각각 y_petal, y_sepal로 정의하겠습니다.

In [3]:
y_petal = iris['petal width (cm)']
y_sepal = iris['sepal width (cm)']

X와 y를 모두 정의했습니다!<br/>
그런데 X의 species 컬럼을 보면 종류가 숫자가 아닌 문자로 나와있습니다.<br/>
컴퓨터는 사람이 사용하는 문자를 알아들을 수 없기 때문에 숫자로 변환해주는 과정이 필요합니다.<br/>
변환 방법에는 여러가지가 있지만, setosa를 1번, virginica는 2번, versicolor를 3번으로 정의해보도록 하겠습니다.

In [4]:
def encode_species2int(species):
    if species == 'setosa':
        return 1
    if species == 'virginica':
        return 2
    if species == 'versicolor':
        return 3

위의 함수는 인자로 붓꽃의 종(species)가 들어오면 숫자를 반환하는 함수 입니다.<br/>
이를 판다스의 시리즈 안에 정의되어 있는 apply라는 메소들을 사용하여 변환하도록 하겠습니다.

In [5]:
species = iris['species'] # 데이터 프레임 내부의 species를 선택(시리즈)
print('species의 데이터 타입:', type(species)) # 데이터 프레임의 컬럼 한 열은 시리즈로 선언되어 있는 것을 확인하실 수 있습니다.
print('\n')
encoded_species = species.apply(encode_species2int)
print('=================species 샘플=================')
print(encoded_species[:5])

species의 데이터 타입: <class 'pandas.core.series.Series'>


=================species 샘플=================
0    1
1    3
2    2
3    2
4    1
Name: species, dtype: int64


apply 메소드는 인자로 받은 함수에 시리즈 원소 하나하나를 넣습니다.<br/>
따라서 encode_species2int 함수에 각각 'setosa', 'virginica', 'versicolor'가 들어가게 되고 이를 1, 2, 3으로 반환하게 되는 것이죠<br/>
이제 정수로 변환한 species를 X의 species 열에 넣겠습니다

In [6]:
X['species'] = encoded_species
X.head()

<ipython-input-6-dff9bf1c1a2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['species'] = encoded_species


,species,sepal length (cm),petal length (cm)
0,1,4.4,1.4
1,3,6.4,4.5
2,2,6.2,4.8
3,2,7.2,6.1
4,1,4.9,1.4


In [7]:
X = np.array(X) # 모든 입력 array로 변환

이제 모든 준비가 끝났습니다.<br/>
본격적으로 너비를 맞춰보도록 하겠습니다!<br/>
지금 주어진 상황은 종류, 꽃받침과 꽃잎의 길이가 주어졌고 이를 통해 꽃받침과 꽃잎의 너비를 알아내야 합니다.

## 예측식 정의
y^ = w1x1 + w2x2 + w3x3 + w4로 표현할 수 있을겁니다.<br/>
y^은 주어진 데이터(종류, 길이)로 예측한 값입니다. x1부터 x3까지는 각각 종류, 꽃받침과 꽃잎의 길이라고 생각하면 됩니다.<br/>
따라서 우리는 x1 부터 x3까지 곱해지는 w1, w2, w3와 절편 w4를 알아내야 합니다.

## 목적식 정의
수식으로 도출된 값을 y^이라 했다면 우리가 알고있는(주어진) 너비 값은 y라고 하겠습니다.<br/>
따라서 우리는 우리가 수식으로 예측한 값과 실제 주어진 값의 차이를 최대한 줄이는 것이 목표입니다.<br/>
이를 수식으로 표현하면 y^ - y = e ~ 0 입니다. e는 'error', 즉 예측한 값과 실제 주어진 값의 차이를 나타냅니다.<br/>
이제 앞의 수식을 제곱하겠습니다.<br/>
<center>1/2(y^-y)^2 = e ~ 0</center><br/>
오차 수식을 제곱하는 이유는 목적식의 볼록한 정도를 유지하고 극값이 최소점인 것을 보장하기 위함입니다.<br/>
극값이 최소점인것을 보장한다면 위의 식을 미분하여 0인 점이 최소점이라는 사실도 보장이 됩니다.

## 미분하여 극값찾기
### 구하고자 하는 값으로 미분
우리가 구하고 싶은 것은 종류, 꼬칭ㅍ의 길이, 꽃받침의 길이와 곱해지는 w1, w2, w3와 절편 w4를 알아내는 것입니다.<br/>
따라서 저 네가지 수로 미분하여 0이 되는 점이 오차가 가장 적은 극점이 될 것입니다.<br/>
자 이제 미분을 해볼까요?
<center>$$\hat y = w1x1 + w2x2 + w3x3 + w4 $$ 이므로<br/>
    $$1/2 * (w1x1 + w2x2 + w3x3 + w4 - y)^2 = e \approx 0 $$ 으로 표현할 수 있습니다.<br/>
~ 중략 ~</center>
이를 코드로 구현해보도록 하겠습니다.<br/>
먼저 절편항을 구하기 위해 1을 전부 추가하겠습니다

In [11]:
ones = np.ones_like(X[:,0]) # 1로만 되어있는 (75, ) 벡터 생성
ones = ones.reshape(75, -1) # 2차원으로 변환 (75, 1)
X = np.concatenate((X,ones), axis=1) 
# 기존 X와 concat 기존 X의 shape = (75, 3) → 변환 뒤 X의 shape (75, 4)

X[:10]

array([[1. , 4.4, 1.4, 1. ],
       [3. , 6.4, 4.5, 1. ],
       [2. , 6.2, 4.8, 1. ],
       [2. , 7.2, 6.1, 1. ],
       [1. , 4.9, 1.4, 1. ],
       [2. , 6.5, 5.8, 1. ],
       [1. , 4.3, 1.1, 1. ],
       [3. , 6.7, 5. , 1. ],
       [3. , 6.8, 4.8, 1. ],
       [3. , 6.6, 4.4, 1. ]])

이제 w를 구하는 식처럼 계산해 보겠습니다. $$W = (X^T*X)^{-1}*X^T*y$$

In [14]:
transpose_doted_X = X.T.dot(X) # X의 전치행렬 X.T와 X 행렬 곱
inversed = np.linalg.inv(transpose_doted_X) # X.T dot X의 역행렬 계산
doted_inv_t = inversed.dot(X.T) # 역행렬과 전치행렬 행렬 곱
weight_petal = doted_inv_t.dot(y_petal) 
# 맞춰야하는 꽃잎 너비와 행렬 곱 → weight 계산
weight_petal

array([-0.06746834, -0.21313084,  0.52330681,  0.59200385])

공식대로 흘러와서 위의 네 개의 w를 얻었습니다.<br/>
잘 동작하는지 확인해보겠습니다.

In [15]:
prediction_petal = X.dot(weight_petal)
error_petal = sum(abs(prediction_petal - y_petal)) / len(prediction_petal)
error_petal

0.14795850337252675

MAE는 0.14795 정도로 나왔네요!<br/>
베이스라인 1의 평균 에러가 0.43이었던 것에 비해 줄어들었습니다<br/>
같은 방식으로 꽃받침의 weight도 구해보겠습니다

In [18]:
weight_sepal = doted_inv_t.dot(y_sepal)
weight_sepal

array([-0.2158848 ,  0.44342083, -0.21595546,  1.67326006])

In [21]:
prediction_sepal = X.dot(weight_sepal)
error_sepal = sum(abs(prediction_sepal - y_sepal)) / len(prediction_sepal)
error_sepal

0.22038467760709277

꽃받침 MAE도 0.2203 정도로 나왔습니다.

위에서 했던 계산 전부를 sklearn의 LinearRegression 클래스에서 자동으로 계산해줍니다!<br/>
sklearn을 써보고 위의 계산과 오차가 얼마나 다른지 확인해보겠습니다.

In [26]:
from sklearn.linear_model import LinearRegression

model_petal = LinearRegression() # 꽃잎의 너비를 예측하는 모델을 선언
model_petal.fit(X[:, :3], y_petal) # X에 추가했던 ones 제거해주세요!

model_sepal = LinearRegression() # 꽃받침의 너비를 예측하는 모델을 선언
model_sepal.fit(X[:, :3], y_sepal) # X에 추가했던 ones 제거해주세요!

# *** cikit learn은 패키지 내부에서 intercept(절편) 계산을 하기 때문에
# 좀전에 절편 계산을 위해 넣어준 ones는 X에서 제외합니다.

LinearRegression()

In [31]:
prediction_sepal_scikit = model_sepal.predict(X[:,:3])
scikit_sepal_error = sum(abs(prediction_sepal_scikit - y_sepal)) / len(prediction_sepal_scikit)
scikit_sepal_error

0.22038467760709296

In [32]:
prediction_petal_scikit = model_petal.predict(X[:,:3])
scikit_petal_error = sum(abs(prediction_petal_scikit - y_petal)) / len(prediction_petal_scikit)
scikit_petal_error

0.14795850337252864

In [35]:
print('직접 구현한 Normal equation 꽃잎 너비 error:', error_petal)
print('sklearn으로 LinearRegression 모델을 사용한 꽃잎 너비 error:', scikit_petal_error)
print('직접 구현한 Normal equation 꽃받침 너비 error:', error_sepal)
print('sklearn으로 LinearRegression 모델을 사용한 꽃받침 너비 error:', scikit_sepal_error)

직접 구현한 Normal equation 꽃잎 너비 error: 0.14795850337252675
sklearn으로 LinearRegression 모델을 사용한 꽃잎 너비 error: 0.14795850337252864
직접 구현한 Normal equation 꽃받침 너비 error: 0.22038467760709277
sklearn으로 LinearRegression 모델을 사용한 꽃받침 너비 error: 0.22038467760709296


직접 구한한 모델과 패키지를 사용한 모델의 오차가 거의 비슷하게 도출되는 것을 확인할 수 있습니다.<br/>
이제 만든 모델로 제출 파일을 만들어보겠습니다.

In [38]:
submission = pd.read_csv('sample_submission.csv') # submission 파일 불러오기
test_set = pd.read_csv('iris_test.csv') # test file 불러오기

In [39]:
test_set['species'] = test_set['species'].apply(encode_species2int)
X = test_set[['species', 'sepal length (cm)', 'petal length (cm)']]

In [40]:
pred_petal = model_petal.predict(X) # 꽃잎 길이 예측
pred_sepal = model_sepal.predict(X) # 꽃받침 길이 예측

In [41]:
submission['sepal width (cm)'] = pred_sepal
submission['petal width (cm)'] = pred_petal

submission.to_csv('second_submission.csv', index = False)